# Transformação de dados - organizando o match history

Do jeito que está nosso dataset, fica difícil de fazer consultas mais elaboradas, como
* Qual campeão foi o mais banido
* Qual o campeão foi o mais escolhido
* Quais campeões tiveram mais vitórias
* Com qual campeão cada player se deu melhor

Então para isso, irei aplicar algumas transformações de dados para que possamos fazer essas consultas futuramente

In [1]:
import pandas as pd

df = pd.read_json('matches.json')

Visualizando nosso dataset

In [2]:
df.head()

,date,patch,team_1,team_2,winner_team,team_1_ban_1,team_1_ban_2,team_1_ban_3,team_1_ban_4,team_1_ban_5,...,team_1_player_1,team_1_player_2,team_1_player_3,team_1_player_4,team_1_player_5,team_2_player_1,team_2_player_2,team_2_player_3,team_2_player_4,team_2_player_5
0,2022-10-31,12.18,Gen.G,DRX,DRX,Heimerdinger,Akali,Aatrox,Renata Glasc,Karma,...,Doran,Peanut,Chovy,Ruler,Lehends,Kingen,Pyosik,Zeka,Deft,BeryL
1,2022-10-30,12.18,Gen.G,DRX,DRX,Heimerdinger,Sylas,Aatrox,Camille,Gragas,...,Doran,Peanut,Chovy,Ruler,Lehends,Kingen,Pyosik,Zeka,Deft,BeryL
2,2022-10-30,12.18,DRX,Gen.G,DRX,Lucian,Singed,Ryze,Gnar,Camille,...,Kingen,Pyosik,Zeka,Deft,BeryL,Doran,Peanut,Chovy,Ruler,Lehends
3,2022-10-30,12.18,Gen.G,DRX,Gen.G,Heimerdinger,Akali,Aatrox,Ashe,Karma,...,Doran,Peanut,Chovy,Ruler,Lehends,Kingen,Pyosik,Zeka,Deft,BeryL
4,2022-10-24,12.18,EDward Gaming,DRX,DRX,Heimerdinger,Aatrox,Caitlyn,Kalista,Varus,...,Flandre,JieJie,Scout,Viper,Meiko,Kingen,Pyosik,Zeka,Deft,BeryL


In [3]:
list(df.columns[5:])

['team_1_ban_1',
 'team_1_ban_2',
 'team_1_ban_3',
 'team_1_ban_4',
 'team_1_ban_5',
 'team_2_ban_1',
 'team_2_ban_2',
 'team_2_ban_3',
 'team_2_ban_4',
 'team_2_ban_5',
 'team_1_pick_1',
 'team_1_pick_2',
 'team_1_pick_3',
 'team_1_pick_4',
 'team_1_pick_5',
 'team_2_pick_1',
 'team_2_pick_2',
 'team_2_pick_3',
 'team_2_pick_4',
 'team_2_pick_5',
 'team_1_player_1',
 'team_1_player_2',
 'team_1_player_3',
 'team_1_player_4',
 'team_1_player_5',
 'team_2_player_1',
 'team_2_player_2',
 'team_2_player_3',
 'team_2_player_4',
 'team_2_player_5']

Separando as colunas de bans, picks e players em datasets diferentes para aplicar transformações

In [4]:
df_transformed_1 = df.iloc[:, 0:14] # melt bans
df_transformed_2 = df.loc[:, ["date", "patch", "team_1", "team_2", "winner_team", 'team_1_pick_1',
 'team_1_pick_2',
 'team_1_pick_3',
 'team_1_pick_4',
 'team_1_pick_5',
 'team_2_pick_1',
 'team_2_pick_2',
 'team_2_pick_3',
 'team_2_pick_4',
 'team_2_pick_5']]
df_transformed_3 = df.loc[:,["date", "patch", "team_1", "team_2", "winner_team", 'team_1_player_1',
 'team_1_player_2',
 'team_1_player_3',
 'team_1_player_4',
 'team_1_player_5',
 'team_2_player_1',
 'team_2_player_2',
 'team_2_player_3',
 'team_2_player_4',
 'team_2_player_5']]

Fazendo a primeira transformação, empilhando os bans

In [5]:
df_1_melted = pd.melt(df_transformed_1, id_vars=["date", "patch", "team_1", "team_2", "winner_team"], value_vars=df_transformed_1.columns[5:], value_name="bans")
df_1_melted = df_1_melted.drop(columns=["variable"])
df_1_melted


,date,patch,team_1,team_2,winner_team,bans
0,2022-10-31,12.18,Gen.G,DRX,DRX,Heimerdinger
1,2022-10-30,12.18,Gen.G,DRX,DRX,Heimerdinger
2,2022-10-30,12.18,DRX,Gen.G,DRX,Lucian
3,2022-10-30,12.18,Gen.G,DRX,Gen.G,Heimerdinger
4,2022-10-24,12.18,EDward Gaming,DRX,DRX,Heimerdinger
...,...,...,...,...,...,...
301,2022-10-13,12.18,T1,Cloud9,T1,Renekton
302,2022-10-13,12.18,T1,Fnatic,T1,Jax
303,2022-10-10,12.18,Cloud9,T1,T1,Renata Glasc
304,2022-10-08,12.18,Fnatic,T1,Fnatic,Pyke


Aplicando a segunda transformação e já unindo com a primeira transformação - empilhando os picks

In [6]:
df_2_melted = pd.melt(df_transformed_2, id_vars=["date", "patch", "team_1", "team_2", "winner_team"],
                      value_vars=df_transformed_2.columns[5:], value_name="picks")
df_2_melted = df_2_melted.drop(columns=["variable"])
df_melted = df_1_melted.join(df_2_melted.iloc[:, -1])
df_melted

,date,patch,team_1,team_2,winner_team,bans,picks
0,2022-10-31,12.18,Gen.G,DRX,DRX,Heimerdinger,Renekton
1,2022-10-30,12.18,Gen.G,DRX,DRX,Heimerdinger,Sejuani
2,2022-10-30,12.18,DRX,Gen.G,DRX,Lucian,Gragas
3,2022-10-30,12.18,Gen.G,DRX,Gen.G,Heimerdinger,Fiora
4,2022-10-24,12.18,EDward Gaming,DRX,DRX,Heimerdinger,Jax
...,...,...,...,...,...,...,...
301,2022-10-13,12.18,T1,Cloud9,T1,Renekton,Kai'Sa
302,2022-10-13,12.18,T1,Fnatic,T1,Jax,Lucian
303,2022-10-10,12.18,Cloud9,T1,T1,Renata Glasc,Kai'Sa
304,2022-10-08,12.18,Fnatic,T1,Fnatic,Pyke,Aphelios


Aplicando a última transformação e renomeando a coluna variable para `team_player` onde será o time do jogador

In [7]:
df_3_melted = pd.melt(df_transformed_3, id_vars=["date", "patch", "team_1", "team_2", "winner_team"], value_vars=df_transformed_3.columns[5:],
                      value_name="players")
df_3_melted

,date,patch,team_1,team_2,winner_team,variable,players
0,2022-10-31,12.18,Gen.G,DRX,DRX,team_1_player_1,Doran
1,2022-10-30,12.18,Gen.G,DRX,DRX,team_1_player_1,Doran
2,2022-10-30,12.18,DRX,Gen.G,DRX,team_1_player_1,Kingen
3,2022-10-30,12.18,Gen.G,DRX,Gen.G,team_1_player_1,Doran
4,2022-10-24,12.18,EDward Gaming,DRX,DRX,team_1_player_1,Flandre
...,...,...,...,...,...,...,...
335,2022-10-13,12.18,T1,Cloud9,T1,team_2_player_5,Zven
336,2022-10-13,12.18,T1,Fnatic,T1,team_2_player_5,Hylissang
337,2022-10-10,12.18,Cloud9,T1,T1,team_2_player_5,Keria
338,2022-10-08,12.18,Fnatic,T1,Fnatic,team_2_player_5,Keria


In [8]:
df_3_melted = df_3_melted.rename(columns={"variable": "team_player"})
df_3_melted

,date,patch,team_1,team_2,winner_team,team_player,players
0,2022-10-31,12.18,Gen.G,DRX,DRX,team_1_player_1,Doran
1,2022-10-30,12.18,Gen.G,DRX,DRX,team_1_player_1,Doran
2,2022-10-30,12.18,DRX,Gen.G,DRX,team_1_player_1,Kingen
3,2022-10-30,12.18,Gen.G,DRX,Gen.G,team_1_player_1,Doran
4,2022-10-24,12.18,EDward Gaming,DRX,DRX,team_1_player_1,Flandre
...,...,...,...,...,...,...,...
335,2022-10-13,12.18,T1,Cloud9,T1,team_2_player_5,Zven
336,2022-10-13,12.18,T1,Fnatic,T1,team_2_player_5,Hylissang
337,2022-10-10,12.18,Cloud9,T1,T1,team_2_player_5,Keria
338,2022-10-08,12.18,Fnatic,T1,Fnatic,team_2_player_5,Keria


Nessa transformação/imputação de dados iremos seguir a seguinte lógica: <br>
* Se team_player tem alguma string "team_1" iremos colocar o nome do time que está na coluna "team_1"
* Se team_player tem alguma string "team_2" colocaremos o nome do time do "team_2"

In [9]:
has_team_1 = df_3_melted["team_player"].str.contains(r'^team_1.*')
has_team_2 = df_3_melted["team_player"].str.contains(r'^team_2.*')

In [10]:
has_team_2

0      False
1      False
2      False
3      False
4      False
       ...  
335     True
336     True
337     True
338     True
339     True
Name: team_player, Length: 340, dtype: bool

Colocando o nome do time de cada player, e verificando se está correto <br>
como por exemplo: Doran pertence a time da Gen.G que está na coluna `team_1` <br>
Kingen pertence a DRX<br>
Porém ainda não definimos os players que estão no `team_2`

In [11]:
df_3_melted.loc[has_team_1, "team_player"] = df_3_melted.team_1

In [12]:
df_3_melted

,date,patch,team_1,team_2,winner_team,team_player,players
0,2022-10-31,12.18,Gen.G,DRX,DRX,Gen.G,Doran
1,2022-10-30,12.18,Gen.G,DRX,DRX,Gen.G,Doran
2,2022-10-30,12.18,DRX,Gen.G,DRX,DRX,Kingen
3,2022-10-30,12.18,Gen.G,DRX,Gen.G,Gen.G,Doran
4,2022-10-24,12.18,EDward Gaming,DRX,DRX,EDward Gaming,Flandre
...,...,...,...,...,...,...,...
335,2022-10-13,12.18,T1,Cloud9,T1,team_2_player_5,Zven
336,2022-10-13,12.18,T1,Fnatic,T1,team_2_player_5,Hylissang
337,2022-10-10,12.18,Cloud9,T1,T1,team_2_player_5,Keria
338,2022-10-08,12.18,Fnatic,T1,Fnatic,team_2_player_5,Keria


Fazendo a mesma transformação para os players que estão nos times da coluna `team_2`

In [13]:
df_3_melted.loc[has_team_2, "team_player"] = df_3_melted.team_2

In [14]:
df_3_melted

,date,patch,team_1,team_2,winner_team,team_player,players
0,2022-10-31,12.18,Gen.G,DRX,DRX,Gen.G,Doran
1,2022-10-30,12.18,Gen.G,DRX,DRX,Gen.G,Doran
2,2022-10-30,12.18,DRX,Gen.G,DRX,DRX,Kingen
3,2022-10-30,12.18,Gen.G,DRX,Gen.G,Gen.G,Doran
4,2022-10-24,12.18,EDward Gaming,DRX,DRX,EDward Gaming,Flandre
...,...,...,...,...,...,...,...
335,2022-10-13,12.18,T1,Cloud9,T1,Cloud9,Zven
336,2022-10-13,12.18,T1,Fnatic,T1,Fnatic,Hylissang
337,2022-10-10,12.18,Cloud9,T1,T1,T1,Keria
338,2022-10-08,12.18,Fnatic,T1,Fnatic,T1,Keria


Verificando o dataset transformado com as colunas bans e picks

In [15]:
df_melted

,date,patch,team_1,team_2,winner_team,bans,picks
0,2022-10-31,12.18,Gen.G,DRX,DRX,Heimerdinger,Renekton
1,2022-10-30,12.18,Gen.G,DRX,DRX,Heimerdinger,Sejuani
2,2022-10-30,12.18,DRX,Gen.G,DRX,Lucian,Gragas
3,2022-10-30,12.18,Gen.G,DRX,Gen.G,Heimerdinger,Fiora
4,2022-10-24,12.18,EDward Gaming,DRX,DRX,Heimerdinger,Jax
...,...,...,...,...,...,...,...
301,2022-10-13,12.18,T1,Cloud9,T1,Renekton,Kai'Sa
302,2022-10-13,12.18,T1,Fnatic,T1,Jax,Lucian
303,2022-10-10,12.18,Cloud9,T1,T1,Renata Glasc,Kai'Sa
304,2022-10-08,12.18,Fnatic,T1,Fnatic,Pyke,Aphelios


Unindo todas as colunas transformadas e vendo nosso dataset final

In [16]:
df_melted = df_melted.join(df_3_melted.iloc[:, -2:])

In [17]:
df_melted

,date,patch,team_1,team_2,winner_team,bans,picks,team_player,players
0,2022-10-31,12.18,Gen.G,DRX,DRX,Heimerdinger,Renekton,Gen.G,Doran
1,2022-10-30,12.18,Gen.G,DRX,DRX,Heimerdinger,Sejuani,Gen.G,Doran
2,2022-10-30,12.18,DRX,Gen.G,DRX,Lucian,Gragas,DRX,Kingen
3,2022-10-30,12.18,Gen.G,DRX,Gen.G,Heimerdinger,Fiora,Gen.G,Doran
4,2022-10-24,12.18,EDward Gaming,DRX,DRX,Heimerdinger,Jax,EDward Gaming,Flandre
...,...,...,...,...,...,...,...,...,...
301,2022-10-13,12.18,T1,Cloud9,T1,Renekton,Kai'Sa,Cloud9,Berserker
302,2022-10-13,12.18,T1,Fnatic,T1,Jax,Lucian,Fnatic,Upset
303,2022-10-10,12.18,Cloud9,T1,T1,Renata Glasc,Kai'Sa,T1,Gumayusi
304,2022-10-08,12.18,Fnatic,T1,Fnatic,Pyke,Aphelios,T1,Gumayusi


Exportando para CSV para futura análise

In [18]:
df_melted.to_csv("match_history_teams.csv", index=False)